In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from collections import defaultdict
import cv2
import matplotlib.pyplot as plt
import random

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
#         print(os.path.join(dirname, filename))
        pass

for dirname, _, filenames in os.walk('/kaggle/input/cmp23-handwritten-males-vs-females'):
    for filename in filenames:
#         print(os.path.join(dirname, filename))
        pass

In [ ]:
def get_files_paths(paths):
    ret = []
    for path in paths:
        for dirname, _, filenames in os.walk(path):
            for filename in filenames:
                ret.append(os.path.join(dirname, filename))
    return ret

In [ ]:
# add icdar when ready
males_paths = ['../Training Dataset/CMP_23/Males']
females_paths = ['../Training Dataset/CMP_23/Females']

males = get_files_paths(males_paths)
females = get_files_paths(females_paths)
        
males_num = len(males)
females_num = len(females)

print(males_num, females_num)


In [ ]:
# max chain code length to calc pdf on
CHAINCODE_LENGTH = 3

In [ ]:
# return chaincodes as list of strings
def chaincodes_from_contours(contours):
    # dx, dy -> direction
    dir = [['0', '1', '2'],
           ['7', '9', '3'],
           ['6', '5', '4']]

#     print(contours)
    conts = ['']*len(contours)
    for idx in range(len(contours)):
        for i in range(len(contours[idx])-1):
#             print( (contours[idx][i+1]))
            dx, dy = (contours[idx][i+1] - contours[idx][i])[0]
            dx += 1
            dy += 1
            conts[idx] += dir[dy][dx]
    return conts

In [ ]:
# pdf of chain code from images!
# [idx][code]
# [idx]['1'] -> pdf of ['1']
# [idx]['01'] -> pdf of ['01']
# [idx]['345'] -> pdf of['345']
def chaincodes_from_images(images, th1 = 100000, th2 = 1000):
    chaincodes = [[]] * len(images)
    for idx in range(len(images)):
        img = cv2.imread(images[idx])
        imgh, imgw = len(img), len(img[0])
        imgarea = imgh * imgw
        
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)        
        ret, thresh = cv2.threshold(img_gray, 150, 255, cv2.THRESH_OTSU)

#         plt.imshow(img)
#         plt.show()
        
#         plt.imshow(thresh)
#         plt.show()
        
        # detect the contours on the binary image using cv2.CHAIN_APPROX_NONE
        contours, _ = cv2.findContours(image=thresh, mode=cv2.RETR_LIST, method=cv2.CHAIN_APPROX_NONE)
        contours_filt = list(filter(lambda cnt: imgarea/th1 <= cv2.contourArea(cnt) <= imgarea/th2, contours))
        
#         img2 = img.copy()
#         cv2.drawContours(img2, contours_filt, -1, (0,255,0), 3)
#         plt.imshow(img2)
#         plt.show()
        
        chaincodes[idx] = chaincodes_from_contours(contours_filt)
    
    return chaincodes

In [ ]:
# idx = 10
# th1 = 100000
# th2 = 100
# img = cv2.imread(females[idx])
# plt.imshow(img)
# female_chaincode = chaincode(females[idx: idx+1], th1, th2)
# print(female_chaincode)

female_chaincodes = chaincodes_from_images(females)
male_chaincodes = chaincodes_from_images(males)

In [ ]:
# [image_index][contour_index]
print(len(female_chaincodes))
print(len(female_chaincodes[0][0][0]))
print(female_chaincodes[0][0])


In [ ]:
def pdf_from_chaincodes(chaincodes):
    pdf = [{} for _ in range(CHAINCODE_LENGTH+1)]
    for dic in pdf:
        dic = defaultdict(lambda: 0, dic)
        
    # clac freq of all substrings with length
    for length in range(1, CHAINCODE_LENGTH+1):
        for cnt_idx in range(len(chaincodes)):
            for i in range(len(chaincodes[cnt_idx]) - length):
                val = chaincodes[cnt_idx][i: i+length]
                pdf[length][val] = pdf[length].get(val, 0) + 1
    return pdf

In [ ]:
# pdf of chain code from images!
# [img_idx][length][code]
# [img_idx][1]['1'] -> pdf of ['1']
# [img_idx][2]['01'] -> pdf of ['01']
# [img_idx][3]['345'] -> pdf of['345']

# length: 0 -> empty!

# list of dicts
female_pdf_chaincodes = [pdf_from_chaincodes(female_chaincodes[idx]) for idx in range(females_num)]
male_pdf_chaincodes = [pdf_from_chaincodes(male_chaincodes[idx]) for idx in range(males_num)]

# img_idx, cnt_idx, i, le = 0, 0, 0, 3
# print(female_pdf_chaincodes[img_idx][cnt_idx][i:i+le])


In [ ]:
def probalize(bad_pdf):
    try:
        a = {key: value/sum(bad_pdf.values()) for key, value in bad_pdf.items()}
        return defaultdict(lambda: 0, a)
    except:
        a = {}
        return defaultdict(lambda: 0, a) # empty dict, avoid div by zero

In [ ]:
# make values from 0 to 1
# NORMALIZED pdf of chain code from images!
# [img_idx][length][code]
# [img_idx][1]['1'] -> pdf of ['1']
# [img_idx][2]['01'] -> pdf of ['01']
# [img_idx][3]['345'] -> pdf of['345']

# length: 0 -> empty!

print(females_num)
print(len(female_pdf_chaincodes))

female_pdf_chaincodes_n = [
    [probalize(female_pdf_chaincodes[img_idx][length]) for length in range(CHAINCODE_LENGTH + 1)]
    for img_idx in range(females_num)]


male_pdf_chaincodes_n = [
    [probalize(male_pdf_chaincodes[img_idx][length]) for length in range(CHAINCODE_LENGTH + 1)]
    for img_idx in range(males_num)]



In [ ]:

print(len(female_pdf_chaincodes_n))
print(female_pdf_chaincodes[0][1]['5'])
print(female_pdf_chaincodes_n[0][1]['5'])

In [ ]:
labels = []
def gen_labels(curr = ''):
    if (len(curr)):
        labels.append(curr)

    if len(curr) < CHAINCODE_LENGTH:
        for i in range(0, 8):
            gen_labels(curr + str(i))
    return
    
labels = []
gen_labels()
# print(labels)

In [ ]:
values = []

female_all_dict = {}
male_all_dict = {}
for label in labels:
    female_all_dict[label] = 0
    male_all_dict[label] = 0


for pdfs in female_pdf_chaincodes_n:
    # [length] -> dict {code: probability}
    for pdf in pdfs:
        female_all_dict.update(pdf)

for pdfs in male_pdf_chaincodes_n:
    # [length] -> dict {code: probability}
    for pdf in pdfs:
        male_all_dict.update(pdf)
        

In [ ]:
assert male_all_dict.keys() == female_all_dict.keys()

In [ ]:

# plt.plot(female_all_dict.keys(), female_all_dict.values(), 'b.')
# plt.show()    


# plt.plot(male_all_dict.keys(), male_all_dict.values(), 'y.')
# plt.show()    


In [ ]:
diff = {}
for label in labels:
    diff[label] = abs(female_all_dict[label] - male_all_dict[label])

# get top 1000
lst_diff = list(diff.items())
lst_diff_s = sorted(lst_diff, key=lambda x: x[1], reverse=True)[:25]

xs = [it[0] for it in lst_diff_s]
ys = [it[1] for it in lst_diff_s]

In [ ]:
plt.rcParams["figure.figsize"] = (17, 6)

plt.plot(xs, ys, 'k.')
plt.show()